In [5]:
import mlflow

from mlflow.tracking import MlflowClient

mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

In [6]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.search_experiments()

[<Experiment: artifact_location='file:///c:/Users/usEr/Documents/GitHub/cpe393-mlflow/mlruns/1', creation_time=1747322598424, experiment_id='1', last_update_time=1747322598424, lifecycle_stage='active', name='mlops_nyc_taxi', tags={}>,
 <Experiment: artifact_location='file:///c:/Users/usEr/Documents/GitHub/cpe393-mlflow/mlruns/0', creation_time=1747322598412, experiment_id='0', last_update_time=1747322598412, lifecycle_stage='active', name='Default', tags={}>]

In [7]:
client.create_experiment(name="NYC Taxi tips experiment")

'2'

In [8]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 100",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [9]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 261efa4975154075b7d9b096677965d5, rmse: 25.7887
run id: bed964269c994e8b8a458a1a03dc0bd1, rmse: 33.9278
run id: 762cb07d53d3446193cd8e2d6c6f68f5, rmse: 59.2400
run id: ea85837bb48742dfb739e70bf6b67c98, rmse: 72.6045


In [10]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [18]:
run_id = "9c6de352d91d4d049eb8b24d5b480b0a"  #insert your run id
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="duration-prediction4")

Registered model 'duration-prediction4' already exists. Creating a new version of this model...


MlflowException: Run with id=9c6de352d91d4d049eb8b24d5b480b0a not found

In [ ]:
#check model version and stages
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

In [ ]:
model_version = 4
new_stage = "Staging"
# Use client.transition_model_version_stage() function to model to staging.

Comparing versions and selecting the new "Production" model
In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
Download the DictVectorizer that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
Preprocess the test set using the DictVectorizer so we can properly feed the regressors.
Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
Based on the results, update the "Production" model version accordingly.
